# VIC - Introduction to Visual Computing

**Instructor:** Maria Vakalopoulou\
**T.A.:** Joseph Boyd\
**Date:** December 8, 2022

## Optical flow 

Optical flow measurs the motion of objects in a scene from a sequence of images. For a given point $p$ in an image scene, the optical flow is the vector $[u, v] = \big[\frac{\partial x}{\partial t}, \frac{\partial y}{\partial t}\big]$ i.e. the speed of the pixel.

**Assumption \#1**: image intensity of point stays the same over time:

$$I(x + dx, y + dy, t + dt) = I(x, y, t)$$

**Assumption \#2**: $dx, dy$ small, and $I$ can be approximated by a first-order Taylor polynomial

$$I(x + dx, y + dy, t + dt) = I(x, y, t) + I_x \cdot dx + I_y \cdot dy + I_t \cdot dt,$$

where $I_x$ denotes $\frac{\partial I}{\partial x}$. Subtracting the two equations, we divide by $\partial t$ and let $\partial t \rightarrow 0$, giving the *optical flow constraint equation*:

$$I_x \cdot u + I_y \cdot v + I_t = 0$$

For the lab exercises use `jupyter` or `colab` (or anything else at your own risk). It can be installed by issuing `pip install jupyter` on Linux after installing pip itself with `apt install python-pip`. Mac and Windows users can find the installation instructions online. The first thing that is needed is to import some libraries, such as `numpy`, `scipy`, `random` and `matplotlib` for visualization. If they are missing you can simply install them with e.g., `pip install numpy`

First let's load the images and display them in sequence:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ipywidgets import interact, widgets
from IPython.display import display

from skimage.io import imread
from skimage.color import rgb2gray
from skimage.transform import rescale

img1 = imread('frame_0_delay-0.5s.png')
img1 = rescale(rgb2gray(img1[..., :3]), scale=0.5)

img2 = imread('frame_1_delay-0.5s.png')
img2 = rescale(rgb2gray(img2[..., :3]), scale=0.5)

frames = [img1, img2]

def f_display(frame):
    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(frames[frame], cmap='gray')

interact(f_display, frame=widgets.IntSlider(min=0, max=1, step=1, value=0));

We define a function for visualising optical flow using the `quiver` function from `matplotlib`. Compile and move on.

In [ ]:
def visualize_flow(img, flow, delta, scale=0.5):

    H, W = img.shape
    flow_i, flow_j = flow

    fig, ax = plt.subplots(figsize=(15, 15))
    ax.imshow(img, cmap='gray')

    Y, X = np.mgrid[delta:H - delta:2 * delta + 1, delta:W - delta:2 * delta + 1]

    ax.quiver(X, Y,                                # position where the flow is computed
              flow_j, flow_i,                      # flow
              np.sqrt(flow_i ** 2 + flow_j ** 2),  # colour the arrows based on this array
              cmap='jet',                          # colour map
              headlength=7,                        # length of the arrowheads
              angles='xy', scale_units='xy',
              scale=scale)

    plt.show()

## Brute force algorithm

We first code a naive brute-force algorithm for optical flow. For each pixel, we form a patch of neighbouring pixels $p$ with radius `r_window`. This is compared with other patches $q$ within a search radius `r_search` using a similarity measure.

### Exercise 1

Write a function `sim_patch` which returns the zero normalized cross-correlations between two input patches ($p$ and $q$),

$$sim_{patch}(p, q) = \frac{\sum_{i, j}(p_{ij} - \bar{p})(q_{ij} - \bar{q})}{\sqrt{\sum_{i, j}(p_{ij} - \bar{p})^2}\sqrt{\sum_{i, j}(q_{ij} - \bar{q})^2}}$$

In [ ]:
def sim_patch(p, q):
    # TODO
    return result

### Exercise 2

Complete the function `optical_flow_brute_force` to compute the optimal displacement for each patch in an image. For efficiency, we don't consider overlapping patches. To speed things up, note that we define a variable to only compute for every `stride`th pixel.

In [ ]:
def optical_flow_brute_force(img1, img2, r_window, r_search):

    """
    Computes optical flow by testing similarity between local windows
    img1: first image in sequence
    img2: second image in sequence
    r_window: radius of window
    r_search: radius of locality in which to search for similar windows
    """

    stride = 2 * r_window + 1
    W, H = img1.shape

    flow_v = np.zeros([len(range(r_window, W - r_window, stride)),
                       len(range(r_window, H - r_window, stride))])
    flow_u = np.zeros([len(range(r_window, W - r_window, stride)),
                       len(range(r_window, H - r_window, stride))])

    # TODO

    return flow_v, flow_u

Now let's visualise. Try different parameters until you are satisfied with the result: 

In [ ]:
r_window = # TODO
r_search = # TODO

flow_v, flow_u = optical_flow_brute_force(img1, img2, r_window, r_search)
visualize_flow(img1, [flow_v, flow_u], r_window)

## Lukas-Kanade method

The optical flow constraint is *underdetermined*, so Lukas-Kanade introduces another constraint:

**Assumption \#3**: motion field, hence optical flow, is constant within a small neighbourhood $W$ of size $N \times N$.

From this we obtain a system of $N^2$ optical flow constraint equations:

$$\underbrace{
\begin{bmatrix}
I_x(1, 1) & I_y(1, 1) \\
\vdots & \vdots \\
I_x(N, 1) & I_y(N, 1) \\
\vdots & \vdots \\
I_x(N, N) & I_y(N, N)
\end{bmatrix}}_{\mathbf{A}}
\underbrace{
\begin{bmatrix}
u \\
v
\end{bmatrix}}_{\mathbf{x}}
= 
\underbrace{\begin{bmatrix}
-I_t(1, 1) \\
\vdots \\
-I_t(N, 1) \\
\vdots \\
-I_t(N, N)
\end{bmatrix}}_{\mathbf{b}}$$

Multipling through by $\mathbf{A}^T$ we obtain,

$$\mathbf{A}^T\mathbf{A}\mathbf{x} = \mathbf{A}^T\mathbf{b},$$

a system of $2$ equations, which moreover may be solved using the method of least squares. Note that,

$$\mathbf{A}^T\mathbf{A} = \begin{bmatrix}
\sum_W I_xI_x & \sum_W I_xI_y \\
\sum_W I_yI_x & \sum_W I_yI_y \\
\end{bmatrix}, \
\mathbf{A}^T\mathbf{b} = \begin{bmatrix}
-\sum_W I_xI_t \\
-\sum_W I_yI_t \\
\end{bmatrix}$$

### Exercise 3

Complete the Lukas-Kanade function below. Again note we define a `stride` variable. Use `numpy.linalg.solve` to solve the linear system. To make things work later, we include a `sigma` parameter to perform a Gaussian blur as a preprocessing step.

In [ ]:
import scipy
from skimage.filters import gaussian, sobel
from numpy.linalg import solve


def lucas_kanade(img1, img2, r_window, sigma=3):

    stride = 2 * r_window + 1
    W, H = img1.shape

    flow_i = np.zeros([len(range(r_window, W - r_window, stride)),
                       len(range(r_window, H - r_window, stride))])
    flow_j = np.zeros([len(range(r_window, W - r_window, stride)),
                       len(range(r_window, H - r_window, stride))])

    img1 = gaussian(img1, sigma)
    img2 = gaussian(img2, sigma)

    # Compute the spatial gradients, I_x, I_y with the predefined sobel function

    I_x = # TODO
    I_y = # TODO

    # Compute the time gradient as the image difference
    I_t = # TODO

    # Using the gradients above, compute the gradient products
    I_xx = # TODO
    I_xy = # TODO
    I_yy = # TODO
    I_xt = # TODO
    I_yt = # TODO

    # We will only compute the flow for disjoint patches
    for i, image_i in enumerate(range(r_window, W - r_window, stride)):
        for j, image_j in enumerate(range(r_window, H - r_window, stride)):

            ref_i = image_i - r_window
            ref_j = image_j - r_window

            # Compute the terms of A^t*A, A^t*b as defined above
            sum_I_xx = # TODO
            sum_I_xy = # TODO
            sum_I_yy = # TODO
            sum_I_xt = # TODO
            sum_I_yt = # TODO

            # Form the matrix A^t*A and vector A^t*b
            ATA = # TODO
            ATb = # TODO

            # Compute det(A^t*A) 
            det = # TODO

            if np.abs(det) == 0:
                # Singular matrix. Bad luck!
                continue

            # Solve the linear system
            sol = # TODO
            flow_i[i, j] = sol[0]
            flow_j[i, j] = sol[1]

    return flow_i, flow_j

Again let's visualise. Try different parameters until you are satisfied with the result: 

In [ ]:
r_window = 7
sigma = 3

flow_i, flow_j = lucas_kanade(img1, img2, r_window, sigma)
visualize_flow(img1, [flow_i, flow_j], r_window)

## Coarse-to-fine algorithm

### Exercise 4

Below is a basic algorithm to iterate the `lucas_kanade` function. Comment it (the function of each block of code should be explained, similar to the above function).

**N.B.**: A coarse-to-fine approach is necessary for good results. To keep things simple we simply blur the image 
with Gaussians of decreasing size. An extension exercise could be to implement the full algorithm

In [ ]:
from scipy.ndimage.interpolation import map_coordinates


def coarse_to_fine(img1, img2, params):
    """
    Simplified coarse-to-fine algorithm
    img1: first image in sequence
    img2: second image in sequence
    params: dictionary of parameters
    """

    d = params['r_window']
    delta = 2 * d + 1

    interp_coo_i_ref = np.repeat(
        np.reshape((np.array(range(0, (img1.shape[0] // delta) * delta))).astype('float'), [1, -1, 1]), 
        (img1.shape[1] // delta) * delta, axis=2)

    interp_coo_j_ref = np.repeat(
        np.reshape((np.array(range(0, (img1.shape[1] // delta) * delta))).astype('float'), [1,  1, -1]),
        (img1.shape[0] // delta) * delta, axis=1)

    interp_coo_ref = np.concatenate([interp_coo_i_ref, interp_coo_j_ref], axis=0)

    interp_coo_i = np.repeat(
        np.reshape((np.array(range(0, (img1.shape[0] // delta) * delta)).astype('float') - d) / delta, [1, -1, 1]), 
        (img1.shape[1] // delta) * delta, axis=2)

    interp_coo_j = np.repeat(
        np.reshape((np.array(range(0, (img1.shape[1] // delta) * delta)).astype('float') - d) / delta, [1, 1, -1]),
        (img1.shape[0] // delta) * delta, axis=1)

    interp_coo = np.concatenate([interp_coo_i, interp_coo_j], axis=0)

    img1_t = img1[:interp_coo.shape[1], :interp_coo.shape[2]]
    img2_t = img2[:interp_coo.shape[1], :interp_coo.shape[2]]

    flow_v_tot = []
    flow_u_tot = []
    full_coo = []

    for t in range(0, params['n_iter']):

        params['sigma'] = params['sigma0'] / np.sqrt(float(t + 1))

        flow_v, flow_u = lucas_kanade(img1_t, img2_t, params['r_window'], params['sigma'])

        if t == 0:
            flow_v_tot = params['step_size'] * flow_v
            flow_u_tot = params['step_size'] * flow_u
        else:
            flow_v_tot += params['step_size'] * flow_v
            flow_u_tot += params['step_size'] * flow_u

        flow_v_full = map_coordinates(flow_v_tot, interp_coo, mode='nearest')
        flow_u_full = map_coordinates(flow_u_tot, interp_coo, mode='nearest')

        full_coo = np.concatenate([
            np.reshape(flow_v_full, [1, flow_v_full.shape[0], flow_v_full.shape[1]]),
            np.reshape(flow_u_full, [1, flow_u_full.shape[0], flow_u_full.shape[1]])],
            axis=0)

        img2_t = map_coordinates(img2, interp_coo_ref + full_coo, mode='nearest')    

        print('[sigma] %.04f [err] %.04f' % (params['sigma'], np.mean((img1_t - img2_t) ** 2)))

    return full_coo, img2_t, img1_t, flow_v_full, flow_u_full

In [ ]:
params = {}

params['n_iter'] = 10
params['step_size'] = 1
params['r_window'] = 5
params['sigma0'] = 3

_, _, _, flow_i, flow_j = coarse_to_fine(img1, img2, params)

flow_i = flow_i[::2 * params['r_window'] + 1, ::2 * params['r_window'] + 1]
flow_j = flow_j[::2 * params['r_window'] + 1, ::2 * params['r_window'] + 1]

visualize_flow(img2, [flow_i, flow_j], params['r_window'], 1)